In [19]:
import datetime
import sys
import os
import openpyxl
import sqlite3
from sqlite3 import Error

reserva=[]

libro = openpyxl.Workbook()
hoja = libro["Sheet"]
hoja.title = "Reporte "

if not os.path.isfile("Registros Reservaciones.db"):
    print("Primera vez de ejecucion")
    try:
        with sqlite3.connect("Registros Reservaciones.db") as conn:
            cursor = conn.cursor()
            cursor.execute("CREATE TABLE IF NOT EXISTS CLIENTE (CLAVE INTEGER PRIMARY KEY, NOMBRE_CLIENTE TEXT NOT NULL);")
            cursor.execute("CREATE TABLE IF NOT EXISTS SALA (CLAVE INTEGER PRIMARY KEY, NOMBRE_SALA TEXT NOT NULL, CUPO_SALA INTEGER NOT NULL);")
            cursor.execute("CREATE TABLE IF NOT EXISTS TURNO (CLAVE INTEGER PRIMARY KEY, TURNO TEXT NOT NULL);")
            cursor.execute("CREATE TABLE IF NOT EXISTS RESERVACION (CLAVE INTEGER PRIMARY KEY, NOMBRE_CLIENTE TEXT NOT NULL, NOMBRE_EVENTO TEXT NOT NULL, FECHA_RESERVA timestamp, TURNO INTEGER NOT NULL, SALA INTEGER NOT NULL,FOREIGN KEY(TURNO) REFERENCES TURNO(CLAVE), FOREIGN KEY(SALA) REFERENCES SALA(CLAVE));")
            print("Tablas creadas exitosamente")

            cursor.execute("INSERT INTO TURNO VALUES(1,'MATUTINO')")
            cursor.execute("INSERT INTO TURNO VALUES(2,'VESPERTINO')")
            cursor.execute("INSERT INTO TURNO VALUES(3,'NOCTURNO')")
    except Error as e:
        print (e)
    except:
        print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
    finally:
        conn.close()

if os.path.isfile("Registros Reservaciones.db"):
    print("Ya existe la base de datos")
    
    while True:
        try:
            print("")
            print("Menu Principal")
            print("")
            print("1. Reservaciones")
            print("2. Reportes.")
            print("3. Registrar una sala ")
            print("4. Registrar un cliente ")
            print("5. Salir")
            print("")
            opcion=input("¿Que opcion del menu deseas? ")
            if opcion == "":
                print("")
                print("No se puede dejar vacio")
                continue      
            opcion_menu=int(opcion) 
            if opcion_menu >5:
                print("")
                print("Opcion no valida")
                continue
        except:
            print("")
            print("Solo se aceptan numeros")
            continue
        else: 
            if opcion_menu == 1 :
                try:
                    print("")
                    print("Menu Reservaciones")
                    print("")
                    print("1. Registrar nueva reservacion")
                    print("2. Modificar descripcion de una reservacion")
                    print("3. Consultar disponibilidad de salas para una fecha")
                    print("4. Eliminar una reservacion")
                    print("5. Salir y volver al menu principal")
                    print("")
                    opciones=input("¿Que opcion del menu deseas? ")
                    if opciones == "":
                        print("") 
                        print("No se puede dejar vacio")
                        continue      
                    opcion_menu_reservacion=int(opciones) 
                    if opcion_menu_reservacion >5:
                        print("")
                        print("Opcion no valida")
                        continue
                except:
                    print("")
                    print("Solo se aceptan numeros")
                    continue
                else:

                    if opcion_menu_reservacion == 1:
                        while True:
                            try:
                                with sqlite3.connect("Registros Reservaciones.db") as conn:
                                    cursor = conn.cursor()
                                    cursor.execute("SELECT * FROM CLIENTE ORDER BY clave")
                                    registros_cliente = cursor.fetchall()
                                    if registros_cliente:
                                        print("Clave\tNombre")
                                        print("*" * 30)
                                        for clave, nombre_cliente in registros_cliente:
                                            print(f"{clave:^6}\t{nombre_cliente}")
                                        print("")
                                    if not registros_cliente:
                                        print("")
                                        print("No hay registros de clientes")
                                        print("")
                                    nombre_cliente=input("Ingrese el nombre completo del cliente : ")
                                    if nombre_cliente == "":
                                        print("")
                                        print("No se puede omitir")
                                        print("")
                                        continue
                                    valores = {"NOMBRE_CLIENTE":nombre_cliente}
                                    cursor.execute("SELECT * FROM CLIENTE WHERE NOMBRE_CLIENTE = :NOMBRE_CLIENTE", valores)
                                    registros_cliente = cursor.fetchall()
                                    if registros_cliente:
                                        for clave,nombre_cliente in registros_cliente:
                                            continue
                                    else:
                                        print("")
                                        print(f"No esta registrado, favor de registrarse")
                                        break

                                    while True:
                                        print("")
                                        nombre_evento=input("Ingresar el nombre del evento : ")
                                        if nombre_evento == "":
                                            print("")
                                            print("No se puede dejar vacio")
                                            continue

                                        while True:
                                            try:
                                                Fecha_actual=datetime.date.today()
                                                print("")
                                                fecha_reservacion=input("Ingrese fecha que deseas reservar (aaaa-mm-dd) : ")
                                                fecha_reserva=datetime.datetime.strptime(fecha_reservacion, "%Y-%m-%d" ).date()
                                                if fecha_reservacion == "":
                                                    print("")
                                                    print("No se puede omitir")
                                                    continue
                                                dias_aprox= fecha_reserva.day - Fecha_actual.day
                                                if dias_aprox <= 2:
                                                    print("")
                                                    print("La reservacion se tiene que hacer con 2 dias de anticipacion")
                                                    continue
                                            except ValueError:
                                                print("")
                                                print("Formato de la fecha es incorrecta")
                                                continue
                                            else:
                                                
                                                while True:
                                                    with sqlite3.connect("Registros Reservaciones.db") as conn:
                                                        cursor = conn.cursor()
                                                        cursor.execute("SELECT * FROM TURNO ORDER BY clave")
                                                        registros_turno = cursor.fetchall()
                                                        if registros_turno:
                                                            print("")
                                                            print("Clave\tTURNO")
                                                            print("*" * 30)
                                                            for clave, turno in registros_turno:
                                                                print(f"{clave:^6}\t{turno}")
                                                            print("")

                                                        turno=input("Ingrese el turno que desea reservar : ")
                                                        if turno == "":
                                                            print("")
                                                            print("No se puede omitir")
                                                            continue
                                                        valores = {"CLAVE":turno}
                                                        cursor.execute("SELECT * FROM TURNO WHERE CLAVE = :CLAVE", valores)
                                                        registros_turno = cursor.fetchall()
                                                        if registros_turno:
                                                            for clave, turno in registros_turno:
                                                                continue
                                                        else:
                                                            print("")
                                                            print("Revisar su opcion de turno seleccionada")
                                                            continue

                                                    while True:
                                                        with sqlite3.connect("Registros Reservaciones.db") as conn:
                                                            cursor = conn.cursor()
                                                            cursor.execute("SELECT * FROM SALA ORDER BY clave")
                                                            registros_sala = cursor.fetchall()

                                                            if registros_sala:
                                                                print("")
                                                                print("Clave\tSALA\tCUPO")
                                                                print("*" * 30)
                                                                for clave, sala,cupo_sala in registros_sala:
                                                                    print(f"{clave:^6}\t{sala:^6}\t{cupo_sala}")
                                                                print("")
                                                            if not registros_sala:
                                                                print("")
                                                                print("No hay salas registradas aun")
                                                                print("")
                                                                break

                                                            sala=input("Ingresa el numero de la sala que desea reservar : ")
                                                            if sala == "":
                                                                print("No se puede omitir")
                                                                continue
                                                            valores = {"CLAVE":sala}
                                                            cursor.execute("SELECT * FROM SALA WHERE CLAVE = :CLAVE", valores)
                                                            registros_sala = cursor.fetchall() 
                                                            if registros_sala:
                                                                for clave, sala, cupo_sala in registros_sala:
                                                                    continue
                                                            else:
                                                                print("")
                                                                print("La opcion no es valida")
                                                                continue
                                                            valores = {"FECHA_RESERVA":fecha_reserva,"TURNO":turno,"SALA":sala}
                                                            cursor.execute("SELECT * FROM RESERVACION WHERE FECHA_RESERVA = :FECHA_RESERVA AND TURNO = :TURNO AND SALA = :SALA", valores)
                                                            registros_reserva = cursor.fetchall()
                                                            if registros_reserva:
                                                                for clave, nombre_cliente, nombre_evento, fecha_reserva, turno, sala in registros_reserva:
                                                                    print("")
                                                                    print("Fecha ocupada, Revisar las salas desocupadas para esa fecha")
                                                                    break
                                                            else:
                                                                try:
                                                                    with sqlite3.connect("Registros Reservaciones.db") as conn:
                                                                        cursor = conn.cursor()
                                                                        valores = {"nombre_cliente":nombre_cliente,"nombre_evento":nombre_evento,"fecha_reserva":fecha_reserva,"turno":turno,"sala":sala}
                                                                        cursor.execute("INSERT INTO RESERVACION (nombre_cliente,nombre_evento,fecha_reserva,turno,sala) VALUES(:nombre_cliente,:nombre_evento,:fecha_reserva,:turno,:sala)", valores)
                                                                        reserva.append((clave,nombre_cliente,nombre_evento,fecha_reserva,turno,sala))
                                                                        print("")
                                                                        print("Registro agregado exitosamente")
                                                                except Error as e:
                                                                    print (e)
                                                                except:
                                                                    print("")
                                                                    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                                                    break
                                                                finally:
                                                                    conn.close()
                                                                break
                                                        break
                                                    break
                                            break
                                        break
                                    break
                            except Error as e:
                                print (e)
                            except:
                                print("")
                                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                break
                            finally:
                                conn.close()

                    if opcion_menu_reservacion == 2:
                        while True:
                            try:
                                with sqlite3.connect("Registros Reservaciones.db") as conn:
                                    cursor = conn.cursor()
                                    cursor.execute("SELECT * FROM RESERVACION ORDER BY clave")
                                    registros_reserva = cursor.fetchall()
                                    if registros_reserva:
                                        print("")
                                        print("{:<15} {:<15} {:<15} {:<20} {:<15} {:<15}".format('CLAVE','NOMBRE','EVENTO','FECHA RESERVA','TURNO','SALA' ))
                                        print("*" * 90)
                                        for clave, nombre_cliente,nombre_evento,fecha_reserva,turno,sala in registros_reserva:
                                            print("{:<15} {:<15} {:<15} {:<20} {:<15} {:<15}".format(clave, nombre_cliente,nombre_evento,fecha_reserva,turno,sala ))

                                    print("")
                                    clave_buscar=input("Ingrese la clave del registro que desea editar : ")
                                    valor={"clave":clave_buscar}
                                    cursor.execute("SELECT * FROM RESERVACION WHERE clave = :clave",valor)
                                    registros_reserva = cursor.fetchall()
                                    if registros_reserva:
                                        print("")
                                        nuevo_nombre=input("Ingrese el nuevo nombre del evento  : ")
                                        valores = {"nombre_evento":nuevo_nombre,"clave":clave_buscar}
                                        cursor.execute("UPDATE RESERVACION SET nombre_evento = :nombre_evento WHERE clave = :clave",valores)
                                        registros_reserva = cursor.fetchall()
                                        print("")
                                        print("Registro Editado exitosamente")
                                        break
                                    else:
                                        if not clave in registros_reserva:
                                            print("")
                                            print("No existe este registro")
                                            continue
                            except Error as e:
                                print (e)
                            except:
                                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                break
                            finally:
                                conn.close()

                    if opcion_menu_reservacion == 3 :
                        encontradas = []
                        disponibles = []
                        while True:
                            try: 
                                print("")
                                fecha_solicitada_normal = input("Ingrese la fecha del evento (aaaa-mm-dd): ")
                                fecha_solicitada = datetime.datetime.strptime(fecha_solicitada_normal, "%Y-%m-%d").date()
                                if fecha_solicitada_normal == "":
                                    print("")
                                    print("No se puede omitir")
                                    continue
                                else:
                                    with sqlite3.connect("Registros Reservaciones.db") as conn:
                                        cursor = conn.cursor()
                                        cursor.execute("SELECT * FROM RESERVACION ORDER BY clave")
                                        registros_reserva = cursor.fetchall()
                                        cursor.execute("SELECT * FROM TURNO")
                                        registros_turno = cursor.fetchall()
                                        cursor.execute("SELECT * FROM SALA")
                                        registros_sala = cursor.fetchall()
                                        if registros_reserva:
                                            for clave,nombre_cliente,nombre_evento,fecha_reservacion,turno,sala in registros_reserva:
                                                fecha_reservacion,turno,sala = (fecha_reservacion,turno,sala)
                                                if fecha_reservacion == fecha_solicitada_normal:
                                                    encontradas.append((sala,turno))
                                            reservas_ocupadas = set(encontradas)
                                            for clave1,sala,cupo_sala in registros_sala:
                                                for clave,turno in registros_turno:
                                                    disponibles.append((sala,turno))
                                            combinaciones_resrvaciones_disponibles = set(disponibles)

                                            salas_turnos_disponibles = sorted(list(combinaciones_resrvaciones_disponibles - reservas_ocupadas))
                                            print("")
                                            print(f"*Salas disponibles para rentar el {fecha_solicitada}*\n")
                                            print("Salas\t\tTurnos")
                                            if registros_sala:
                                                for sala,turno in salas_turnos_disponibles:
                                                    print(f"{sala}\t\t{turno}")
                                                break
                            except ValueError:
                                print("")
                                print("Formato de la fecha es incorrecta")
                                continue     
                            except Error as e:
                                print (e)
                            except:
                                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                break
                            finally:
                                conn.close()

                    if opcion_menu_reservacion == 4:
                        while True:
                            try:
                                with sqlite3.connect("Registros Reservaciones.db") as conn:
                                    cursor = conn.cursor()
                                    cursor.execute("SELECT * FROM RESERVACION ORDER BY clave")
                                    registros_reserva = cursor.fetchall()

                                    if registros_reserva:
                                        print("")
                                        print("{:<15} {:<15} {:<15} {:<20} {:<15} {:<15}".format('CLAVE','NOMBRE','EVENTO','FECHA RESERVA','TURNO','SALA' ))
                                        print("*" * 90)
                                        for clave_reserva, nombre_cliente,nombre_evento,fecha_reserva,turno,sala in registros_reserva:
                                            print("{:<15} {:<15} {:<15} {:<20} {:<15} {:<15}".format(clave_reserva, nombre_cliente,nombre_evento,fecha_reserva,turno,sala ))

                                    print("")
                                    registro_eliminar=input("Ingrese la clave del registro que desea eliminar : ")
                                    with sqlite3.connect("Registros Reservaciones.db") as conn:
                                        cursor = conn.cursor()
                                        valor = {"clave":registro_eliminar}
                                        cursor.execute("SELECT * FROM RESERVACION WHERE clave = :clave",valor)
                                        registros_reserva = cursor.fetchall()
                                        if registros_reserva:
                                            print("")
                                            print("{:<15} {:<15} {:<15} {:<20} {:<15} {:<15}".format('CLAVE','NOMBRE','EVENTO','FECHA RESERVA','TURNO','SALA' ))
                                            print("*" * 90)
                                            for clave_reserva, nombre_cliente,nombre_evento,fecha_reserva,turno,sala in registros_reserva:
                                                print("{:<15} {:<15} {:<15} {:<20} {:<15} {:<15}".format(clave_reserva, nombre_cliente,nombre_evento,fecha_reserva,turno,sala ))

                                        else:
                                            if not clave_reserva in registros_reserva:
                                                print("")
                                                print("No existe esta clave")
                                                continue

                                        print("")
                                        asegurar =input("Esta seguro de eliminar el registro S/N : ")
                                        if asegurar == "":
                                            print("")
                                            print("No se puede dejar vacio")
                                        if (not asegurar.upper() in "SN"):
                                            print("")
                                            print("Opcion no valida")
                                        if (asegurar.upper() == "S"):
                                            cursor.execute("DELETE FROM RESERVACION WHERE clave = :clave",valor)
                                            print("")
                                            print("Registro Borrado con exito")
                                            break
                                        elif (asegurar.upper() == "N"):
                                            print("")
                                            print("1. Desea regresar al menu principal")
                                            print("2. Eliminar otro registro")
                                            print("")
                                            regresar = input("Ingrese su opcion : ")
                                            if regresar == "1":
                                                break
                                            elif regresar == "2":
                                                continue  
                            except Error as e:
                                print (e)
                            except:
                                print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                                break
                            finally:
                                conn.close()

                    if opcion_menu_reservacion == 5:
                        continue

        if opcion_menu == 2:
            try:
                print("")
                print("Menu Reporte")
                print("")
                print("1. Reporte en pantalla de reservaciones para una fecha")
                print("2. Exportar reporte tabular en Excel")
                print("3. Salir y volver al menu principal")
                print("")
                opciones=input("¿Que opcion del menu deseas? ")
                if opciones == "":
                    print("") 
                    print("No se puede dejar vacio")
                    continue      
                opcion_menu_reporte=int(opciones) 
                if opcion_menu_reporte >3:
                    print("")
                    print("Opcion no valida")
                    continue
            except:
                print("")
                print("Solo se aceptan numeros")
                continue
            else:
                if opcion_menu_reporte == 1:
                    while True:
                        try:
                            print("")
                            fecha_solicitada_normal = input("Ingrese la fecha del evento (aaaa-mm-dd): ")
                            fecha_solicitada = datetime.datetime.strptime(fecha_solicitada_normal, "%Y-%m-%d").date()
                            if fecha_solicitada_normal == "":
                                print("")
                                print("No se puede omitir")
                            else:
                                with sqlite3.connect("Registros Reservaciones.db") as conn:
                                    cursor = conn.cursor()
                                    cursor.execute("SELECT * FROM RESERVACION ORDER BY clave")
                                    registros_reserva = cursor.fetchall()
                                    print("")
                                    print("**"*48)
                                    print("*" + " "*23 + f"REPORTE DE RESERVACIONES PARA EL DÍA {fecha_solicitada}" + " "*23 + "*")
                                    print("**"*48)
                                    print("{:<23} {:<23}  {:<23} {:<23}".format('SALA','NOMBRE','EVENTO', 'TURNO' ))
                                    print("**"*48)
                                    if registros_reserva:
                                        for clave,nombre_cliente,nombre_evento,fecha_reservacion,turno,sala in registros_reserva:
                                            if fecha_reservacion == fecha_solicitada_normal:
                                                print(f"{sala}\t\t\t{nombre_cliente}\t\t\t{nombre_evento}\t\t\t{turno}")
                                    print("")
                                    print("*"*40 + " FIN DEL REPORTE " + "*"*40)  
                                    break
                        except ValueError:
                            print("")
                            print("Formato de la fecha es incorrecta")
                            continue 
                        except Error as e:
                            print (e)
                        except Exception:
                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                            break
                        finally:
                            conn.close()

                if opcion_menu_reporte == 2:
                    while True:
                        try:
                            print("")
                            fecha_solicitada_normal = input("Ingrese la fecha del evento (aaaa-mm-dd): ")
                            fecha_solicitada = datetime.datetime.strptime(fecha_solicitada_normal, "%Y-%m-%d").date()
                            if fecha_solicitada_normal == "":
                                print("")
                                print("No se puede omitir")
                                continue
                            else:
                                with sqlite3.connect("Registros Reservaciones.db") as conn:
                                    cursor = conn.cursor()
                                    cursor.execute("SELECT * FROM RESERVACION ORDER BY clave")
                                    registros_reserva = cursor.fetchall()
                                    if registros_reserva:
                                        hoja["A1"].value = f"REPORTE DE RESERVACIONES PARA EL DÍA {fecha_solicitada}"
                                        hoja["A2"].value = "SALA"
                                        hoja["B2"].value = "NOMBRE"
                                        hoja["C2"].value = "EVENTO"
                                        hoja["D2"].value = "TURNO"

                                        for clave,nombre_cliente,nombre_evento,fecha_reservacion,turno,sala in registros_reserva:
                                            if fecha_reservacion == fecha_solicitada_normal:
                                                elementos_sala = [(sala,nombre_cliente,nombre_evento,turno)]
                                                for elemento in elementos_sala:
                                                    hoja.append(elemento)
                                        libro.save("Reporte.xlsx")
                                        print("")
                                        print("Libro creado exitosamente")
                                    break         
                        except ValueError:
                            print("")
                            print("Formato de la fecha es incorrecta")
                            continue
                        except Error as e:
                            print (e)
                        except Exception:
                            print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                            break
                        finally:
                            conn.close()

                if opcion_menu_reporte ==3:
                    continue

        if opcion_menu == 3:
            while True:
                try:
                    print("")
                    nombre_sala=input("Ingrese el nombre de la sala : ")
                    if nombre_sala == "":
                        print("")
                        print("No se puede omitir")
                        continue
                    else:
                        print("")
                        cupo_sala=input("Ingresar el cupo de la sala : ")
                        if cupo_sala == "":
                            print("")
                            print("No se puede omitir")
                            continue
                        cupo = int(cupo_sala)  
                        if cupo== 0 :
                            print("")
                            print("Debe de ser mayor a 0")
                            continue 
                        else:
                            with sqlite3.connect("Registros Reservaciones.db") as conn:
                                cursor = conn.cursor()
                                valores = {"nombre_sala":nombre_sala,"cupo_sala":cupo_sala}
                                cursor.execute("INSERT INTO SALA (nombre_sala, cupo_sala) VALUES(:nombre_sala, :cupo_sala)", valores)
                                print("")
                                print("Registro agregado exitosamente")
                            break
                except ValueError:
                    print("")
                    print("No es dato numerico")
                    continue
                except Error as e:
                    print (e)
                except:
                    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                    break
                finally:
                    conn.close()

        if opcion_menu == 4:
            while True:
                try:
                    print("")
                    nombre_cliente=input("Ingrese el nombre completo del cliente : ")
                    if nombre_cliente == "":
                        print("")
                        print("No se puede omitir")
                        continue
                    else:
                        with sqlite3.connect("Registros Reservaciones.db") as conn:
                            cursor = conn.cursor()
                            valores = {"nombre_cliente":nombre_cliente}
                            cursor.execute("INSERT INTO CLIENTE (nombre_cliente) VALUES(:nombre_cliente)", valores)
                            print("")
                            print("Registro agregado exitosamente")   
                            break     
                except Error as e:
                    print (e)
                except:
                    print(f"Se produjo el siguiente error: {sys.exc_info()[0]}")
                    break
                finally:
                    conn.close()
    
        if opcion_menu == 5 :
            print("")
            print("Gracias, Vuelva pronto")
            break 

Ya existe la base de datos

Menu Principal

1. Reservaciones
2. Reportes.
3. Registrar una sala 
4. Registrar un cliente 
5. Salir

¿Que opcion del menu deseas? 2

Menu Reporte

1. Reporte en pantalla de reservaciones para una fecha
2. Exportar reporte tabular en Excel
3. Salir y volver al menu principal

¿Que opcion del menu deseas? 3

Menu Principal

1. Reservaciones
2. Reportes.
3. Registrar una sala 
4. Registrar un cliente 
5. Salir

¿Que opcion del menu deseas? 5

Gracias, Vuelva pronto
